In [3]:
import requests
import json
import pandas as pd

In [5]:
# test token
token = 'EAACEdEose0cBAMLdZBNFZBK6g6v7VQ70R1nQipq0qZBYqblZAZBsbCbVmfxfpZApZCTo98Ctd6vOPWnbgPLt6yTqhD7EGWIaBKd6ZBwk4gzWYZBlbkejSvY3yLgZCfKSTZCghZASQdFYhwjCZC0ZAZB6N02592S6tJ9AZCIrBYFqfJCk8NqgMnZCu60RSXbEzvafOtCOH9DtPHWKGTljAbgZDZD'
res = requests.get("https://graph.facebook.com/v2.11/me?access_token=%s" % (token))
print(res.text)

{"name":"Alicia Lee","id":"844389662238733"}


In [7]:
pages = ('BaiChiGongZhu')

# 粉絲頁基本資料
for page in pages:
    info = '?fields=id,name,about,category,likes,link,talking_about_count,website'
    res = requests.get('https://graph.facebook.com/%s%s&access_token=%s' % (page, info, token))
    #res.text
    #json.loads(res.text)['id']

    
    pg1 = [page,                         # 縮名，string
      json.loads(res.text)['id'],        # 唯一碼，numeric
      json.loads(res.text)['name'],      # 粉專名稱，string
      json.loads(res.text)['likes'],     # 粉絲數，numeric
      json.loads(res.text)['talking_about_count'],   # 討論數，numeric
      json.loads(res.text)['category'],  # 分類，string
      json.loads(res.text)['about']      # 介紹，string
          ]     
    

In [22]:
pg2 = ['test01',
       9187298370419,
       'beautiful',
       128793,
       1987239,
       'test',
       'test test test']

In [29]:
pg1[1]

'363734540324168'

In [72]:
# 將兩個dict組合成dataframe
df = pd.DataFrame([pg1,pg2])

In [80]:
# 為dataframe欄位命名
df = df.rename(columns={0:'PID',1:'ID',2:'NAME',3:'LIKES',4:'TALKA',5:'TYPE',6:'ABOUT'})

In [78]:
# 增加一row到dataframe中
df = df.append([pg1,pg2], ignore_index=True)

In [81]:
df

,PID,ID,NAME,LIKES,TALKA,TYPE,ABOUT
0,BaiChiGongZhu,363734540324168,白癡公主,1011178,20809,Producer,舒服合作信箱87gongzhu@gmail.com\n
1,test01,9187298370419,beautiful,128793,1987239,test,test test test
2,test01,9187298370419,beautiful,128793,1987239,test,test test test
3,BaiChiGongZhu,363734540324168,白癡公主,1011178,20809,Producer,舒服合作信箱87gongzhu@gmail.com\n
4,test01,9187298370419,beautiful,128793,1987239,test,test test test


In [ ]:
pages = ('BaiChiGongZhu')

# 貼文基本統計
for page in pages:
    info = '?fields=feed.limit(10){from, id, type, created_time, message, source, shares}'
    period = '&since=1513296000&until=1514678400'            # 近兩週，2017/12/15-12/30
    res = requests.get('https://graph.facebook.com/%s%s%s&access_token=%s' % (page, info, period, token))
    #res.text
    #json.loads(res.text)['id']
    
    feed = [page,                            # 粉專縮名
           json.loads(res.text)['from'],     # 粉專名稱
           json.loads(res.text)['id'],       # 文章唯一碼
           json.loads(res.text)['type'],     # 文章類型
           json.loads(res.text)['created_time'],  # 貼文時間
           json.loads(res.text)['message'],  # 貼文內容
           json.loads(res.text)['source'], # 貼文內容的外部來源
           json.loads(res.text)['shares'], # shares
            
            # 讚
            # reaction_love
            # reaction_haha
            # reaction_sad
            # reaction_happy
            # reaction_angry
           ]
    
    

In [32]:
# 測試ID 
page = 'BaiChiGongZhu'
#page = 'teemomustdie'
#page = '168161663222590'

# 只算LIKE數
#commend = '/feed?fields=message,likes.limit(1).summary(true)'
# 某段時間內的貼文
#commend = '/feed?fields=message,likes.limit(1),created_time&until=1514462400&since=1513771200&limit=100'
# 粉絲團基本資料
commend = '?fields=id,name,about,category,likes,link,talking_about_count,website'

res = requests.get("https://graph.facebook.com/%s%s&access_token=%s" %(page, commend, token))
print(res.text)

{"id":"363734540324168","name":"\u767d\u7661\u516c\u4e3b","about":"\u8212\u670d\u5408\u4f5c\u4fe1\u7bb187gongzhu\u0040gmail.com\n","category":"Producer","likes":1010895,"link":"https:\/\/www.facebook.com\/BaiChiGongZhu\/","talking_about_count":18458,"website":"http:\/\/pics.ee\/87yt"}


In [ ]:
# 粉絲數
commend = '?fields=fan_count'
# 粉絲頁縮名
commend = '?fields=page_token'


In [ ]:
# 近十篇貼文
commend = 'BaiChiGongZhu?fields=feed.limit(10)'
# 貼文分享自其他來源 {parenet_id}
commend = 'BaiChiGongZhu?fields=feed.limit(10){parent_id}'

# 貼文的ID {id}
# 貼文中tag其他人 {message_tags}
# 貼文中影片的名稱 {name}
# 貼文類型 {icon}  | /icon/post.gif 一般貼文(可能附有照片或影片連結或新聞連結) | /icon/video.gif 影片貼文 | NA 分享貼文
# 貼文類型 {status_type} | added_photos | added_video | mobile_status_update | shared_story |
# 貼文類型 {type} | photo | video | link |
# 原貼文的內容文字 {description} 如果是分享自其他來源就會是原貼文的文字
# 粉絲團貼文內容 {message} 分享的貼文，附加的文字說明在此
# 貼文時間 {created_time}
# 貼文唯一碼 {created_time}
# 哪個粉絲團的貼文 {from} 查單一粉絲團時用不到
# 貼文的預設靜態圖 {full_picture}
# 貼文的導向連結 {link} 可以是內部或外部
# 貼文內容的外部來源 {source}
# 分享次數 {shares}
# 最後加上時間區間 &since=1480849200&until=1480856400  使用timestamp

In [ ]:
# 某貼文的互動數，所有reaction
commend = '363734540324168_1763402230357385?fields=reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry)'
# 文章觀看數 ?fields=seen 沒有權限
# 文章分析  ?fields=insights 沒有權限
